#################################################
##      뉴스기사 키워드 추출 및 테마 추천       ##
#################################################

0. 라이브러리 임포트

In [37]:
# -*- coding: utf-8 -*-
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import pickle
import re
import itertools

1. 뉴스기사 선언

In [38]:
article = """
한국전력공사가 적자 해소를 명분으로 농사용 전기요금 체계 개편을 구상 중인 가운데 밑그림이 될 연구 결과가 최근 나왔다. 농민이 많이 쓰는 농사용(을) 요금을 인상하자는 게 골자로, 구상이 현실화하면 채산성이 악화돼 시름에 잠긴 우리 농업이 타격을 입을 수 있다는 우려가 나온다.

본지가 최근 문대림 더불어민주당 의원(제주갑)을 통해 입수한 한전의 ‘농사용 전기요금 체계 개편 방향 수립 연구 보고서’에 따르면 한전의 농사용 전기요금 개편의 주요 타깃은 농사용(을)이 될 것으로 보인다. 이 보고서는 지난해 연구용역을 수탁한 에너지경제연구원이 작성해 한전에 제출한 것이다.

농사용 전기는 양곡 생산을 위한 양·배수 펌프 및 수문 조작에 사용되는 농사용(갑)과 육묘, 작물 재배, 축산, 농산물 저온보관 등에 적용되는 농사용(을)로 구분된다. 농사용(을)은 다시 저압과 고압으로 나뉘는데, 보고서는 농사용(을) 저압 요금을 현재 수준으로 유지하되 계절요금제를 도입할 것을 제안했다.

계절요금제는 현재 농사용(을) 고압에만 적용된다. 봄·가을(1㎾당 60.2원)보다 여름·겨울(62.2원)에 높은 전력량요금을 책정하는 방식으로, 이런 체계가 농사용(을) 저압에도 도입되면 전력가동률이 높은 여름·겨울철 상당수 농가의 전기요금이 오를 가능성이 있다. 2022년 기준 농사용 전기 소비자의 94.9%가 농사용(을) 저압을 사용했다.

보고서는 농사용(을) 고압의 계절별 전력량요금 배율을 확대하고 요금 수준도 높이자고 했다. 특히 계약전력 300㎾(킬로와트) 이상 소비자에 대해선 산업용 수준의 요금을 적용하자고 제안했다. 현재 산업용(을) 요금은 계절·시간에 따라 1㎾당 93.1∼242.7원에 이른다.

이밖에 보고서는 농사용(갑)의 경우 현행 요금 수준의 단일요금제를 유지하자고 했다. 농사용(을) 요금 인상에 따른 소비자 보호 방안으로는 소비자가 과거보다 전기 사용량을 일정 수준 이상 줄이면 차액 일부를 현금으로 돌려주는 ‘에너지 캐시백’ 도입을 제시했다. 정부엔 태양광발전 보급 확대 및 에너지 효율 사업을 통해 농업부문의 전력 소비 절감을 유도해달라고 주문했다.

농업계는 애당초 답이 정해진 연구였다면서 쓴 입맛을 다신다. 농사용 전기요금 체계 개편에 불을 지핀 건 지난해초 전남 구례에서 발생한 사건이었다. 한전이 저온저장고에 김치를 보관했다는 이유로 구례의 농가에 과징금을 부과하자 농업계가 현실적인 제도 개선책을 마련해달라고 요구하면서 연구는 시작됐다. 하지만 연구는 이런 배경을 고려하지 않고 한전 입맛에 맞춘 ‘농사용 전기 혜택 축소’ 쪽으로 결론을 냈다.

이 과정에서 농사용 전기가 한전 적자의 주범이라는 논리가 되풀이됐다. 보고서는 “원가회수율이 낮은 농사용 전기요금 지원액은 한전의 적자로 운영되고 있다”고 밝혔다. 그러면서 “농사용 전기요금 지원에 정부 예산 투입을 공식화해야 한다”는 입장도 피력했다. 하지만 2022년 기준 전체 전력 사용량의 4%가량만 차지하는 농사용 전기를 적자 주범으로 내모는 건 옹색하다는 비판이 농업계에선 나온다.

최범진 한국후계농업경영인중앙연합회 정책조정실장은 “경영비가 급등하는 가운데 전기요금이 인상되면 농가경영은 위기를 맞을 수 있다”면서 “농사용(을) 요금 인상이 불가피하다면 농사용(갑) 사용처를 필수 농업생산 활동으로 확대하는 방안 등을 검토해야 한다”고 주장했다.

저렴한 요금 때문에 농촌에 전기 사용 증가 흐름이 확산하고, 특히 혜택이 대농에 집중된다는 점도 농사용 전기요금 인상 근거로 다시 활용됐다. 이에 대해 국회입법조사처는 “이런 결과는 에너지원의 가격뿐 아니라 농업 규모화, 스마트와 시설 농업의 발달, 농업법인의 증가 등이 두루 영향을 끼친 것으로 추측된다”고 밝혔다.

한전은 “보고서는 참고용이며, 전기요금 체계 개편 권한은 산업통상자원부에 있다”는 입장이지만 한전이 같은 논리를 반복·강화하며 여론 작업을 하고 있어 농업계는 향후 개편 논의를 불안하게 지켜볼 수밖에 없는 실정이다. 전기요금 체계 개편은 한전이 약관을 개정하고 산자부가 기획재정부와 협의를 거쳐 최종 확정하는 방식으로 이뤄진다.

문 의원은 “연구 결과가 현실화하면 이미 경영 위기에 봉착한 농어민이 고사 상태에 빠질 수 있다”면서 “여당 대표는 취약계층 전기요금 지원을 외치고, 한전은 물밑에서 농사용 전기요금 인상을 준비하는 모습은 이율배반적이라고 할 수밖에 없다”고 지적했다. 
"""

2. 텍스트 전처리

In [39]:
article = article.replace('\n', '')
split_article = re.split(r'(\s*[a-zA-Z]+(?:[^\w\s]+[a-zA-Z]+)*\s*)', article)       #   한글과 영어 나누기
split_article = [part.strip() for part in split_article if part.strip()]            #   불필요한 공백 지우기

okt = Okt()                         #   형태소 분석기 Okt
nouns=[]
for sentence in split_article:
    sen = okt.nouns(sentence)
    if len(sen) != 0:               #   명사를 리스트에 추가
        nouns.extend(sen)
    else:
        nouns.append(sentence)      #   영어도 리스트에 추가(형태소 분석 후 길이가 0이면 영어로 판단)

text = ' '.join(nouns)              #   리스트를 하나의 문자열로

print(f'전처리 후 뉴스 기사 : {text}')

전처리 후 뉴스 기사 : 한국전력 공사 적자 해소 명분 농 사용 전기요금 체계 개편 구상 중인 가운데 밑그림 연구 결과 최근 농민 농 사용 요금 인상 게 골자 구상 현실 채 산성 악화 시름 우리 농업 타격 입 수 우려 최근 대림 민주당 의원 제주 갑 통해 입수 한전 농 사용 전기요금 체계 개편 방향 수립 연구 보고서 한전 농 사용 전기요금 개편 주요 타깃 농 사용 이 것 이 보고서 지난해 용역 수탁 에너지 연구원 작성 한전 제출 것 농 사용 전기 양곡 생산 위 양 배수 펌프 및 수문 조작 사용 농 사용 갑 과 육묘 작물 재배 축산 농산물 보관 등 적용 농 사용 로 구분 농 사용 은 다시 저압 고압 보고서 농 사용 저압 요금 현재 수준 유지 계절 금제 도입 것 제안 계절 금제 현재 농 사용 고압 적용 봄 가을 당 여름 겨울 전력량 요금 책정 방식 체계 농 사용 저압 도입 전력 률 여름 겨울철 상당수 농가 전기요금 가능성 기준 농 사용 전기 소비자 농 사용 저압 사용 보고서 농 사용 고압 계절 별 전력량 요금 배율 확대 요금 수준 높이 자고 계약 전력 킬로와트 이상 소비자 대해 선 산업 용 수준 요금 적용 제안 현재 산업 용 요금 계절 시간 당 이 보고서 농 사용 갑 의 경우 현행 요금 수준 단일 금제 유지 농 사용 요금 인상 소비자 보호 방안 소비자 과거 전기 용량 일정 수준 이상 차액 일부 현금 에너지 캐시 도입 제시 정부 태양광 발전 보급 확대 및 에너지 효율 사업 통해 농업부 문의 전력 소비 절감 유도 주문 농업 애 당초 답 정해진 연구 입맛 농 사용 전기요금 체계 개편 불 지핀 건 해초 전남 구례 발생 사건 한전 장고 김치 보관 이유 구례 농가 과징금 부과 농업 현실 제도 개선 책 마련 요구 연구 시작 연구 배경 고려 한전 입맛 농 사용 전기 혜택 축소 쪽 결론 이 과정 농 사용 전기 한전 적자 범 논리 되풀이 보고서 가회 율 농 사용 전기요금 액 한전 적자 운영 고 농 사용 전기요금 지원 정부 예산 투입 공식 입장 피 기준 전체 전력 용량 가량 차지 농

3. N-Gram 생성 및 SBERT

In [40]:
n_gram_range = (1,2)
word_vectorize = CountVectorizer(ngram_range=n_gram_range).fit([text])   #   1 ~ 2 글자로 N-Gram 생성
n_gram_words = word_vectorize.get_feature_names_out()
print(f'N-Gram 개수 : {len(n_gram_words)}')
print(f'N-Gram 다섯개 출력 : {n_gram_words[:5]}')

model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')  #   Sentence Transformers
article_embedding = model.encode([article])
n_gram_embeddings = model.encode(n_gram_words)
print(f'SBERT 분석 후 뉴스 기사 : {article_embedding}')
print(f'SBERT 분석 후 N-Gram : {n_gram_embeddings}')

N-Gram 개수 : 587
N-Gram 다섯개 출력 : ['가격' '가격 농업' '가능성' '가능성 기준' '가량']


c:\Users\G-10\anaconda3\envs\tensor\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


SBERT 분석 후 뉴스 기사 : [[-1.82420775e-01 -2.57038951e-01  7.07125127e-01  1.23314321e-01
   7.27116130e-04 -1.74101181e-02 -1.91261649e-01 -7.70111561e-01
   5.34605347e-02 -6.41670465e-01  1.77726164e-01  5.80364466e-01
  -2.43063554e-01 -1.66203454e-01 -2.79148310e-01 -1.09717334e-02
   1.93167105e-01 -2.11805135e-01  1.56460270e-01 -6.70219958e-02
  -7.69764930e-02 -3.39404404e-01  1.04135513e+00  5.23664892e-01
   3.02445292e-01 -4.34024066e-01  2.16542363e-01  8.97443891e-01
  -6.59733936e-02  1.57856554e-01  2.84976542e-01 -1.71466023e-01
  -3.97847146e-01 -2.50871718e-01 -3.66217166e-01  1.03143620e+00
  -1.36998609e-01  2.38476306e-01  1.71522275e-01 -5.07032648e-02
  -5.10699868e-01 -6.93346336e-02 -2.93882668e-01 -1.64023906e-01
  -7.47244298e-01 -1.83959175e-02 -4.88883674e-01  4.44218993e-01
  -2.85103202e-01 -1.11732550e-01 -1.29568800e-01  4.18338418e-01
  -8.89546424e-03 -4.49931383e-01 -3.29711437e-01  4.25981194e-01
  -6.94477141e-01 -3.41158867e-01 -1.08761072e+00 -4.0481

4. Max Sum Similarity

In [41]:
def max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n, variety):
    #   뉴스 기사와 N-Gram의 유사도
    distances = cosine_similarity(article_embedding, n_gram_embeddings)
    #   N-Gram들 사이의 유사도
    distances_candidates = cosine_similarity(n_gram_embeddings, n_gram_embeddings)

    #   뉴스 기사와 유사도가 높은 N-Gram을 variety개 가져오기
    words_idx = list(distances.argsort()[0][-variety:])
    words_vals = [n_gram_words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    #   그 중 유사도가 가장 낮은 조합 찾기
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

news_keywords = max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n=5, variety=10)
print(f'뉴스 키워드 5개 : {news_keywords}')

뉴스 키워드 5개 : ['전기요금 인상', '보급 확대', '개선 마련', '농업 타격', '약관 개정']


5. 테마 가져오기

In [42]:
#       딕셔너리에서 key 값 가져오기
with open('D:/TJ_FInal_Project/KDJ/News_Keyword_Theme_Recommend.py/data/theme_dict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

keys = loaded_dict.keys()
keys_list = list(keys)

new_keys_list = [re.sub(r'[^a-zA-Z0-9가-힣\s]', ' ', key) for key in keys_list]     #   테마 전처리
key_embeddings = model.encode(new_keys_list)                                        #   테마  SBERT 분석

6. 코사인 거리 기반 & Max Sum Similarity 테마 추천

In [43]:
top_n = 5
distances = cosine_similarity(article_embedding, key_embeddings)
cosine_recommend = [keys_list[index] for index in distances.argsort()[0][-top_n:]]

# maxsum_recommend = max_sum_sim(article_embedding, key_embeddings, keys_list, top_n=3, variety=30)

article = article.replace('.', '.\n')
print(f'뉴스 기사 : \n{article}')
print(f'뉴스 키워드 5개 : {news_keywords}')

print(f'\n뉴스 기사와 연관 되어 있는 테마 5개')
for i in range(0,5):
    print(f'{i+1}. {cosine_recommend[-i]} : {loaded_dict[cosine_recommend[-i]]}')

# print(f'\n뉴스 기사와 연관될 수 있는 테마 3개')
# maxsum_index = 1
# for maxsum in maxsum_recommend:
#     print(f'{maxsum_index}. {maxsum} : {loaded_dict[maxsum]}')
#     maxsum_index += 1

뉴스 기사 : 
한국전력공사가 적자 해소를 명분으로 농사용 전기요금 체계 개편을 구상 중인 가운데 밑그림이 될 연구 결과가 최근 나왔다.
 농민이 많이 쓰는 농사용(을) 요금을 인상하자는 게 골자로, 구상이 현실화하면 채산성이 악화돼 시름에 잠긴 우리 농업이 타격을 입을 수 있다는 우려가 나온다.
본지가 최근 문대림 더불어민주당 의원(제주갑)을 통해 입수한 한전의 ‘농사용 전기요금 체계 개편 방향 수립 연구 보고서’에 따르면 한전의 농사용 전기요금 개편의 주요 타깃은 농사용(을)이 될 것으로 보인다.
 이 보고서는 지난해 연구용역을 수탁한 에너지경제연구원이 작성해 한전에 제출한 것이다.
농사용 전기는 양곡 생산을 위한 양·배수 펌프 및 수문 조작에 사용되는 농사용(갑)과 육묘, 작물 재배, 축산, 농산물 저온보관 등에 적용되는 농사용(을)로 구분된다.
 농사용(을)은 다시 저압과 고압으로 나뉘는데, 보고서는 농사용(을) 저압 요금을 현재 수준으로 유지하되 계절요금제를 도입할 것을 제안했다.
계절요금제는 현재 농사용(을) 고압에만 적용된다.
 봄·가을(1㎾당 60.
2원)보다 여름·겨울(62.
2원)에 높은 전력량요금을 책정하는 방식으로, 이런 체계가 농사용(을) 저압에도 도입되면 전력가동률이 높은 여름·겨울철 상당수 농가의 전기요금이 오를 가능성이 있다.
 2022년 기준 농사용 전기 소비자의 94.
9%가 농사용(을) 저압을 사용했다.
보고서는 농사용(을) 고압의 계절별 전력량요금 배율을 확대하고 요금 수준도 높이자고 했다.
 특히 계약전력 300㎾(킬로와트) 이상 소비자에 대해선 산업용 수준의 요금을 적용하자고 제안했다.
 현재 산업용(을) 요금은 계절·시간에 따라 1㎾당 93.
1∼242.
7원에 이른다.
이밖에 보고서는 농사용(갑)의 경우 현행 요금 수준의 단일요금제를 유지하자고 했다.
 농사용(을) 요금 인상에 따른 소비자 보호 방안으로는 소비자가 과거보다 전기 사용량을 일정 수준 이상 줄이면 차액 일부를 현금으로 돌려주는 ‘에너지 캐시백’ 도입을 제